In [1]:
#callbacks are way to customize the behaviour of model doing training or evaluation

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [3]:
(ds_train, ds_test), ds_info = tfds. load (
    "mnist", #it is name that is in catelog of link above
    split=["train", "test"],
    shuffle_files=True, #tf datasets store something in records in multiple files, for easy stream, loading simultaneously while training
                        # so it is for shuffles dataponits in batch so not sequential
    as_supervised=True, #it return tuple (image,label) otherwise it returns dict
    with_info=True, #for getting info

)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [4]:
def normalize_image(image, label):
  return tf.cast(image, tf.float32)/255.0, label

In [5]:
AUTOTUNE=tf.data.experimental.AUTOTUNE #tf allow us to give this to set automatically
BATCH_SIZE=32

ds_train=ds_train.map(normalize_image, num_parallel_calls=AUTOTUNE) # in normalize function these is no sequence so
#process can be done paralelly
ds_train= ds_train.cache() #it keep next data in memory so it will be fast
ds_train=ds_train.shuffle(ds_info.splits['train'].num_examples) #we can give 1000 so it will see 1000 datat points but
#it depends on records in a file so take from that
ds_train=ds_train.batch(BATCH_SIZE)
ds_train=ds_train.prefetch(AUTOTUNE) #it keeps 64 examples ready when running on gpu the current one

In [6]:
ds_test=ds_test.map(normalize_image, num_parallel_calls=AUTOTUNE)
ds_test=ds_test.batch(128)
ds_test=ds_test.prefetch(AUTOTUNE)

In [23]:
model = keras. Sequential([
    keras. Input ((28, 28, 1)),

#   data_augmentation,

    layers.Conv2D(4, 3, padding="same", activation="relu"),
    layers.Flatten(),
    layers. Dense (64, activation="relu"),
    layers. Dense (10),
])

In [11]:
#save callbacks

save_callback = keras.callbacks.ModelCheckpoint(
    'checkpoint/', save_weights_only=True, monitor='accuracy', save_best_only=False,
)

In [15]:
#how epoch progresses we want to change that

def scheduler (epoch, lr):
  if epoch < 2:
    return lr

  else:
    return lr * 0.99 #decreasing lr

lr_scheduler = keras.callbacks.LearningRateScheduler(scheduler,verbose=1)

In [16]:
model.compile(optimizer=keras.optimizers.Adam(),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=["accuracy"],
)
model.fit(ds_train,
          epochs=5,
          verbose=2,
          callbacks=[save_callback, lr_scheduler])

model.evaluate(ds_test, verbose=2)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/5
1875/1875 - 21s - loss: 0.0780 - accuracy: 0.9765 - lr: 0.0010 - 21s/epoch - 11ms/step

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/5
1875/1875 - 24s - loss: 0.0490 - accuracy: 0.9854 - lr: 0.0010 - 24s/epoch - 13ms/step

Epoch 3: LearningRateScheduler setting learning rate to 0.0009900000470224768.
Epoch 3/5
1875/1875 - 17s - loss: 0.0352 - accuracy: 0.9888 - lr: 9.9000e-04 - 17s/epoch - 9ms/step

Epoch 4: LearningRateScheduler setting learning rate to 0.000980100086890161.
Epoch 4/5
1875/1875 - 18s - loss: 0.0251 - accuracy: 0.9919 - lr: 9.8010e-04 - 18s/epoch - 10ms/step

Epoch 5: LearningRateScheduler setting learning rate to 0.0009702991275116801.
Epoch 5/5
1875/1875 - 18s - loss: 0.0182 - accuracy: 0.9942 - lr: 9.7030e-04 - 18s/epoch - 9ms/step
79/79 - 3s - loss: 0.0629 - accuracy: 0.9810 - 3s/epoch - 35ms/step


[0.06294097751379013, 0.9810000061988831]

In [24]:
#we can create custom callbacks chnaging functionality according to your wants

class CustomCallback(keras.callbacks.Callback):

  def on_epoch_end (self, epoch, logs=None) : #at every epoch end it print logs keys
    print (logs.keys ())
    if logs.get ("accuracy") > 0.98:
      print ("Accuracy over 90%, quitting training")
      self.model.stop_training = True

In [25]:
model.compile(optimizer=keras.optimizers.Adam(),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=["accuracy"],
)
model.fit(ds_train,
          epochs=10,
          verbose=2,
          callbacks=[save_callback, lr_scheduler, CustomCallback()])

model.evaluate(ds_test, verbose=2)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/10
dict_keys(['loss', 'accuracy', 'lr'])
1875/1875 - 20s - loss: 0.2088 - accuracy: 0.9395 - lr: 0.0010 - 20s/epoch - 11ms/step

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/10
dict_keys(['loss', 'accuracy', 'lr'])
1875/1875 - 18s - loss: 0.0719 - accuracy: 0.9778 - lr: 0.0010 - 18s/epoch - 9ms/step

Epoch 3: LearningRateScheduler setting learning rate to 0.0009900000470224768.
Epoch 3/10
dict_keys(['loss', 'accuracy', 'lr'])
Accuracy over 90%, quitting training
1875/1875 - 18s - loss: 0.0470 - accuracy: 0.9853 - lr: 9.9000e-04 - 18s/epoch - 10ms/step
79/79 - 1s - loss: 0.0638 - accuracy: 0.9799 - 1s/epoch - 17ms/step


[0.06382085382938385, 0.9799000024795532]